Sistema de Recomendación con KNN -> MovieLens

**PAQUETES REQUERIDOS**

In [ ]:
import pip
pip.main(['install', 'scikit-surprise'])

In [279]:
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise import accuracy
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

**ANALISIS EXPLORATORIO (EDA)**

Primero hacemos una exploración descriptiva de nuestros datos.

**CARGA DE DATOS**

In [280]:
# Cargar el dataset MovieLens 100k
# Estructura (usuario, ítem, rating, timestamp)
data = Dataset.load_builtin('ml-100k')

<h2>Mostrar los datos</h2>

In [ ]:
# Convertir el conjunto de datos en un dataframe
df = pd.DataFrame(data.raw_ratings, columns=['user_id', 'item_id', 'rating', 'timestamp'])
df.head(5)

<h2>Visualizar descripción de los datos</h2>

In [ ]:
for col in df.columns:
    print(col,' has nulls =>', df['user_id'].isnull().any())

In [ ]:
df.describe(include='all')

<h2>Convertir datos no númericos a numéricos</h2>

In [284]:
df['user_id'] = pd.to_numeric(df['user_id'])
df['item_id'] = pd.to_numeric(df['item_id'])

<h2>Mostrar los datos nueva vez</h2>

In [ ]:
df.describe(include='all')

<h2>Eliminar repeticiones y datos nulos</h2>

In [ ]:
null_columns = df.columns[df.isnull().any()]
if null_columns.size > 0:
    print('Eliminando valores nulos...', end=' ')
    df.dropna(inplace=True)
    print('Valores nulos eliminados')
else:
    print('No hay valores nulos')


In [ ]:
are_duplicates = df.duplicated()
if True in are_duplicates.values:
    print('Eliminando duplicados...', end=' ')
    df.drop_duplicates(inplace=True)
    print('Valores duplicados eliminados')
else:
    print('No hay duplicados')

<h2>Distribución de los ratings</h2>

In [ ]:
sns.countplot(x='rating', data=df)
plt.title('Distribución de Ratings en MovieLens 100k')
plt.show()

<h2>Separar los datos en trainset y testset</h2>

In [289]:
trainset, testset = train_test_split(data, test_size=0.25)

<h2>Definir parámetros para los modelos</h2>

In [290]:
options = {}
for opt in ['cosine', 'pearson', 'msd', 'pearson_baseline']:
    optx = {
        #cosine,pearson,msd,sd
        'name': opt, 
        # True si la similitud es entre usuarios, False si es entre ítems 
        'user_based': True
    }
    options.update({opt:optx})

knn_cosine = KNNBasic(k=50, min_k=10, sim_options=options.get('cosine'))
knn_pearson = KNNBasic(k=50, min_k=10, sim_options=options.get('pearson'))
knn_msd = KNNBasic(k=50, min_k=10, sim_options=options.get('msd'))
knn_pearson_baseline = KNNBasic(k=50, min_k=10, sim_options=options.get('pearson_baseline'))

<h2>Entrenar modelos KNN</h2>

In [ ]:
knn_cosine.fit(trainset=trainset)
knn_pearson.fit(trainset=trainset)
knn_msd.fit(trainset=trainset)
knn_pearson_baseline.fit(trainset=trainset)

knn_models = {'cosine':knn_cosine, 'pearson':knn_pearson, 'msd':knn_msd, 'pearson_baseline':knn_pearson_baseline}

<h2>Hacer predicciones en los modelos</h2>

In [292]:
predictions_cosine = knn_cosine.test(testset)
predictions_pearson = knn_pearson.test(testset)
predictions_msd = knn_msd.test(testset)
predictions_pearson_baseline = knn_pearson_baseline.test(testset)


<h2>Calcular el MAE(Mean Absolute Error) y RMSE(Root Mean Squared Error)</h2>

In [293]:
rmses = []
maes = []
for prediction in [predictions_cosine, predictions_pearson, predictions_msd, predictions_pearson_baseline]:
    rmse = accuracy.rmse(prediction, verbose=False)
    mae = accuracy.mae(prediction, verbose=False)
    rmses.append(rmse)
    maes.append(mae)

<h2>Mostrar el MAE y RMSE de cada configuración de modelo</h2>

In [ ]:
df_result = pd.DataFrame(data=[(rmse, mae) for rmse,mae in zip(rmses, maes)], columns=['MAE','RMSE'], index=['cosine','pearson','msd','pearson_baseline'])
df_result

<code>Un MAE bajo indica que, en promedio, las predicciones del modelo están cerca de los valores reales. Esto es generalmente deseable y significa que el modelo tiene un buen rendimiento en la predicción.</code>

<code>Un RMSE bajo indica que las predicciones del modelo son generalmente cercanas a los valores reales, lo que es deseable. Un RMSE bajo sugiere que el modelo tiene un buen desempeño en la predicción.</code>

<strong>Para nuestro caso se evidencia que el mejor <code>MAE/RMSE</code> lo tiene el modelo que aplica el <code>MSD</code></strong>

<h2>Obtener la matriz de similaridad</h2>

In [ ]:
knn_cosine.compute_similarities()
similarity_matrix = knn_models.get('msd').sim

In [ ]:
# Convertir a DataFrame para facilitar la visualización
sim_df = pd.DataFrame(similarity_matrix)

# Visualizar la matriz de similitud con un mapa de calor
plt.figure(figsize=(10, 8))
plt.imshow(sim_df, cmap='hot', interpolation='nearest')
plt.colorbar()
plt.title('Matriz de Similitud')
plt.xlabel('Ítems')
plt.ylabel('Ítems')
plt.show()

<h2>Comparación de RMSE: Entrenamiento vs Prueba (MSD)</h2>

In [ ]:
# Hacer predicciones en el conjunto de entrenamiento
train_predictions = knn_models.get('msd').test(trainset.build_testset())

# Hacer predicciones en el conjunto de prueba
test_predictions = knn_models.get('msd').test(testset)

train_rmse = accuracy.rmse(train_predictions, verbose=False)
test_rmse = accuracy.rmse(test_predictions, verbose=False)

labels = ['Entrenamiento', 'Prueba']
rmse_values = [train_rmse, test_rmse]

plt.bar(labels, rmse_values, color=['red', 'green'])
plt.title('Comparación de RMSE: Entrenamiento vs Prueba (MSD)')
plt.ylabel('RMSE')
plt.ylim(0, max(rmse_values) + 0.5)
plt.show()

<h2>Recomendación de una película</h2>

In [318]:
def recomendacion(user_id, knn_model, n_recommendations=5):
    # Obtener las películas que el usuario ya ha calificado
    user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]
    rated_items = [item[0] for item in user_ratings]

    # Obtener todas las películas en el dataset
    all_items = trainset.all_items()

    # Predecir calificaciones para ítems no calificados
    predictions = []
    for item_id in all_items:
        if item_id not in rated_items:
            pred = knn_model.predict(str(user_id), trainset.to_raw_iid(item_id))
            predictions.append((item_id, pred.est))

    # Ordenar las predicciones y tomar las "n_recommendations" mejores
    predictions.sort(key=lambda x: x[1], reverse=True)
    top_recommendations = predictions[:n_recommendations]

    return top_recommendations

In [319]:
#pred = knn_models.get('msd').predict('186', '302')
#pred

user_id = str(186)
recommendations = recomendacion(user_id=user_id, knn_model=knn_models.get('msd'), n_recommendations=5)

print("Recomendaciones para el usuario {}: ".format(user_id))
for item_id, est_rating in recommendations:
    print(f"ITEM_ID: {item_id}, Calificación estimada: {est_rating:.2f}")

Recomendaciones para el usuario 186: 
ITEM_ID: 243, Calificación estimada: 4.57
ITEM_ID: 283, Calificación estimada: 4.51
ITEM_ID: 301, Calificación estimada: 4.48
ITEM_ID: 673, Calificación estimada: 4.47
ITEM_ID: 172, Calificación estimada: 4.46


<strong>Ejemplo de salida</strong><br><br><span>user: 186        item: 302        r_ui = None   est = 3.98   {'actual_k': 50, 'was_impossible': False}</span>

<table>
    <thead>
        <th>user</th>
        <th>item</th>
        <th>r_ui</th>
        <th>est</th>
        <th>Detail</th>
    </thead>
    <tbody>
        <tr>
            <td><span>El ID del usuario para el cual estás haciendo la predicción.</span></td>
            <td><span>El ID del ítem (película) para el cual estás haciendo la predicción de rating.</span></td>
            <td><span>Este campo representa el valor real del rating que el usuario le dio a este ítem. En este caso es None porque probablemente no se conoce el valor real (puede que sea una predicción para un ítem que el usuario no ha calificado).</span></td>
            <td><span>Este es el valor estimado de la predicción, lo que significa que el modelo predice que el usuario 186 probablemente le daría una calificación de 3.98 a la película 302.</span></td>
            <td>
                <table>
                    <thead>
                        <th><span>actual_k</span></th>
                        <th><span>was_impossible</span></th>
                    </thead>
                    <tbody>
                        <tr>
                            <td>
                                <span>Indica que el modelo utilizó 50 vecinos cercanos (similar a lo que estableciste en el parámetro k)</span>
                            </td>
                            <td>
                                <span>Esto indica que la predicción fue posible, es decir, el modelo pudo encontrar suficientes vecinos cercanos para hacer la predicción. Si hubiera sido True, significaría que no pudo encontrar suficientes vecinos.</span>
                            </td>
                        </tr>
                    </tbody>
                </table>
            </td>
        </tr>
    </tbody>
</table>